<a href="https://colab.research.google.com/github/amulyagupta1278/Coding-Projects/blob/Deep-Learning/Convolution%20NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision

In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F

Downloading Dataset

In [ ]:
mnist_train_images = torchvision.datasets.MNIST(root='', download = True, train=True, transform=torchvision.transforms.ToTensor())
mnist_valid_images = torchvision.datasets.MNIST(root='', download = True, train=False, transform=torchvision.transforms.ToTensor())
len(mnist_train_images), len(mnist_valid_images)


# mnist_train_images.__len__ = lambda : "hello"

(60000, 10000)

In [ ]:
len(mnist_train_images)

60000

Dataset has to be converted to dataloaders

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(mnist_train_images, batch_size=16)
valid_dl = DataLoader(mnist_valid_images, batch_size=32)

In [ ]:
x,y=next(iter(train_dl))
x.shape

torch.Size([16, 1, 28, 28])

## Creating our model

Convolutional Neural Network

# Convolutional Neural 

Read about Padding and Strides in CNNs


Strides are used to save computation

Padding is done to retain information within the image

In [ ]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func=func
    
    def forward(self,x): return self.func(x)
    def __repr__(self): return f"Lambda({str(self.func).split()[1]})"

def flatten(x): return x.view(x.shape[0],-1) #16,1,28,28 -> (16,1x28x28)
def mnist_preprocess(x): 
    x= x.view(-1,1,28,28) 
    x = F.normalize(x)
    return x


def get_cnn_model(num_classes=10): #helper function which returns the model class
    assert type(num_classes)==int
    layers=[Lambda(mnist_preprocess), 
            nn.Conv2d(1,8,5,stride=2,padding = 2), #input = 16x1x28x28, output = 16x8x28x28
            nn.ReLU(),
            nn.Conv2d(8,16,kernel_size=3,stride=2,padding=1), # #output = 16x16x28x28
            nn.ReLU(),
            nn.Conv2d(16,32,3,2,1), #output 16x32x_x_
            nn.ReLU(),
            nn.Conv2d(32,32,3,2,1), #output  16 x 32 x H x W
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1), # output 16x32x1x1
            Lambda(flatten),    #output 16x32
            nn.Linear(32,num_classes) #output = 16x10
            ]
    return nn.Sequential(*layers)

#nn.Sequential is basically a class given by PyTorch, exactly the same the model we built before
# we had to 
# initialize the super().__init__
# declare each layer as a module
# define an explicit forward function
#take care of how GPU operations will work    
    

In [ ]:
model = get_cnn_model(10)
model

Sequential(
  (0): Lambda(mnist_preprocess)
  (1): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (2): ReLU()
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): ReLU()
  (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (6): ReLU()
  (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (8): ReLU()
  (9): AdaptiveAvgPool2d(output_size=1)
  (10): Lambda(flatten)
  (11): Linear(in_features=32, out_features=10, bias=True)
)

In [ ]:
x,y=next(iter(train_dl))
x.shape, model(x).shape

(torch.Size([16, 1, 28, 28]), torch.Size([16, 10]))

Model construction is done, now we define loss function


Homework - understand the working of this loss function

In [ ]:
def logsumexp(x):
  m=x.max(-1,keepdim=True).values
  return m+ (x-m).exp().sum(-1,keepdim=True)
def log_softmax(x): return x -logsumexp(x)
def nll(pred,target):
  log_sm_pred=log_softmax(pred)
  return -log_sm_pred[range(log_sm_pred.shape[0]),target].mean()

Optimizer

In [ ]:
class optimizer():
  def __init__(self, params, lr=0.1): self.params, self.lr = list(params), lr
  def step(self):
    with torch.no_grad():
      for p in self.params: 
          p-=self.lr*p.grad

  def zero_grad(self):
    for p in self.params: p.grad.data.zero_()

In [ ]:
def accuracy(pred,targ): return (torch.argmax(pred,dim=1)==targ).float().mean()

Extra note: To run the model on GPU, 

1. Put x,y on GPU
2. Put model on GPU


x = x.cuda()

y=y.cuda()

model = model.cuda()

In [ ]:
torch.cuda.is_available()

True

Learner function

In [ ]:
# def mnist_preprocess(x):
#     x= x.view(-1,1,28,28) 
#     x = F.normalize(x)
#     return x

class Learner():
    def __init__(self, train_dl, valid_dl,model=get_cnn_model(), optimizer=None):
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.model = model
        if torch.cuda.is_available(): self.model = self.model.cuda()
        self.opt = optimizer(self.model.parameters) if not optimizer else optimizer #show working with another optimizer

    def fit(self, epochs=1, lr=None, loss_func=nll):
        if lr: self.opt.lr = lr
        for epoch in range(epochs):
            for x,y in train_dl:
                if torch.cuda.is_available(): x,y=x.cuda(), y.cuda()
                    
                
                loss=nll(self.model(x),y)
                loss.backward()
                self.opt.step()
                self.opt.zero_grad()
            
            
            #validation accuracy calculation
            acc=0.
            i=0
            for x,y in valid_dl:
                if torch.cuda.is_available(): x,y=x.cuda(), y.cuda()
                acc+=accuracy(self.model(x),y)
                i+=1
            print(f"epoch{epoch+1}: validation accuracy: {acc/i}")
    
    def predict(self,x): return torch.argmax(self.model(x),dim=1)




In [ ]:
model = get_cnn_model(num_classes=10)
opt=optimizer(model.parameters())
learn = Learner(train_dl,valid_dl, model=model, optimizer=opt)
learn.fit(10)

More Complex Networks

Complex dataset

COmplex Neural Networks

Cat vs Dog Images

In [ ]:
!pip install jmd_imagescraper >./tmp

In [ ]:
from jmd_imagescraper.core import *
from pathlib import Path


root = Path().cwd()/"images"
duckduckgo_search(root,"cat", "cat", max_results=200)
duckduckgo_search(root,"dog","dog",max_results=200)

Duckduckgo search: cat


Duckduckgo search: dog


[PosixPath('/content/images/dog/001_e0f68140.jpg'),
 PosixPath('/content/images/dog/002_0b5f7111.jpg'),
 PosixPath('/content/images/dog/003_11978de8.jpg'),
 PosixPath('/content/images/dog/004_85590eea.jpg'),
 PosixPath('/content/images/dog/005_31af9c1d.jpg'),
 PosixPath('/content/images/dog/006_927fb252.jpg'),
 PosixPath('/content/images/dog/007_9acf6f88.jpg'),
 PosixPath('/content/images/dog/008_01241e54.jpg'),
 PosixPath('/content/images/dog/009_3a352698.jpg'),
 PosixPath('/content/images/dog/010_48d7ea9f.jpg'),
 PosixPath('/content/images/dog/011_354b8f24.jpg'),
 PosixPath('/content/images/dog/012_be1239ba.jpg'),
 PosixPath('/content/images/dog/013_77da8369.jpg'),
 PosixPath('/content/images/dog/014_33ec9451.jpg'),
 PosixPath('/content/images/dog/015_79a9f6e7.jpg'),
 PosixPath('/content/images/dog/016_c659d23b.jpg'),
 PosixPath('/content/images/dog/017_eb4405f2.jpg'),
 PosixPath('/content/images/dog/018_7b54753f.jpg'),
 PosixPath('/content/images/dog/019_8ee8da98.jpg'),
 PosixPath('

In [ ]:
!cd images/cat && ls

001_99bf44c5.jpg  051_1ac5535f.jpg  101_20f176f1.jpg  151_a0615e4a.jpg
002_b430063d.jpg  052_c192156f.jpg  102_830fad30.jpg  152_6d0dd3b2.jpg
003_840a9a33.jpg  053_9a64280f.jpg  103_70c9955f.jpg  153_240899a8.jpg
004_e4f3dff7.jpg  054_805a784d.jpg  104_711c7cc5.jpg  154_6ed521bf.jpg
005_4c3cd5eb.jpg  055_b21e7707.jpg  105_0e9d20ea.jpg  155_81ebab31.jpg
006_be547fb6.jpg  056_1252efcf.jpg  106_5c8f6f02.jpg  156_8ae42daf.jpg
007_c066efb6.jpg  057_a8d867ec.jpg  107_c58f3751.jpg  157_8c732c79.jpg
008_7a6098f2.jpg  058_ffa347e6.jpg  108_16af297e.jpg  158_c607f2f7.jpg
009_68559625.jpg  059_d6195447.jpg  109_25581a62.jpg  159_256d19d6.jpg
010_ac24e6da.jpg  060_7292b39d.jpg  110_a235eaf9.jpg  160_be4cb193.jpg
011_c6c433c2.jpg  061_5788dde4.jpg  111_478f01b9.jpg  161_7ffd9196.jpg
012_9816c045.jpg  062_98a3b032.jpg  112_72e76f95.jpg  162_2b8fb6d1.jpg
013_00a83cc0.jpg  063_d3b92114.jpg  113_5b8803f0.jpg  163_b548f298.jpg
014_0799e4bd.jpg  064_03a53d50.jpg  114_9ce60873.jpg  164_50119cfe.jpg
015_f8

In [ ]:
from jmd_imagescraper.imagecleaner import *
display_image_cleaner(root)

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…

In [ ]:
from torch.utils.data import DataLoader, Dataset, random_split
import os
from PIL import Image

class CatvsDogDataset(Dataset):
    def __init__(self,root='images',transforms=None):
        self.classes = {v:i for i,v in enumerate(os.listdir(root))}
        self.transforms = [lambda x: x] if not transforms else transforms

        self.x=[]
        self.y=[]

        for p,d,f in os.walk(root):
            for file in f:
                if file.endswith('.jpg'):
                    self.x.append(Image.open(Path(p)/file))
                    self.y.append(self.classes[Path(p).stem])


    def __len__(self): return len(self.x)
    def __getitem__(self,i): 
        return (self.apply_tfms(self.x[i],self.transforms),self.y[i])
    


    def apply_tfms(self,x,tfm_list):
        for tfm in tfm_list: x= tfm(x)
        return x

In [ ]:
data = CatvsDogDataset(transforms = [torchvision.transforms.ToTensor(),
                                     torchvision.transforms.CenterCrop(size=300)])

In [ ]:
def split_ds_by_pct(dataset, pct=0.8):
    assert 0.<pct<1., "pct should be (a floating point) between 0 and 1"
    return random_split(dataset,[int(pct*len(dataset)), (len(dataset) - int(pct*len(dataset)) )])

train_ds,val_ds = split_ds_by_pct(data)
len(train_ds), len(val_ds)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(val_ds, batch_size=2*64, shuffle=False)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
x,y = next(iter(train_dl))
x.shape

Advanced model: resnet18

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)
resnet18 = resnet18.cuda() if device else resnet18
resnet18


In [ ]:
resnet18.conv1

In [ ]:
resnet18.fc

In [ ]:
resnet18.fc = nn.Linear(512, 2) #because we are doing a 2 label classification


In [ ]:
def get_resnet(classes =2,pretrained=True, device='cuda:0'):
    resnet18 = torchvision.models.resnet18(pretrained=True)
    resnet18 = resnet18.cuda() if device=='cuda' else resnet18

    resnet18.fc = nn.Linear(512, classes)

    # resnet18.fc = resnet18.fc.cuda() if device=='cuda' else resnet18.fc

    return resnet18.cuda() if torch.cuda.is_available() else resnet18

In [ ]:
x,_ = next(iter(train_dl))
x=x.cuda()
resnet18(x).shape

In [ ]:
def preprocess(x):
    x=torch.nn.functional.normalize(x)
    return x

In [ ]:
class Learner():
  def __init__(self,train_dl,valid_dl,model,optimizer=None): #remove default value of model
    self.train_dl=train_dl
    self.valid_dl=valid_dl
    self.model=model
    self.opt=optimizer(self.model.parameters()) if not optimizer else optimizer

  def fit(self,epochs=1, bs=64,lr=None,loss_func=nll):
    lr=self.opt.lr if not lr else lr
    for epoch in range(epochs):
      for x,y in train_dl:
        x,y=x.cuda(),y.cuda()
        x=preprocess(x)

        loss = nll(self.model(x),y)
        loss.backward()
        self.opt.step()
        self.opt.zero_grad()
      acc=0.
      i=0

      for x,y in list(valid_dl):
        x,y = x.cuda(),y.cuda()
        x=preprocess(x)         # mnist_preprocess --> process
        acc+=accuracy(self.model(x),y)
        i+=1
      print(f"epoch {epoch+1}: validation accuracy: {acc/i}")
  
  def predict(self,x): return torch.argmax(self.model(x),dim=1)



model=get_resnet(classes =2,device='cuda')
# model=get_cnn_model(num_classes=2)
opt=optimizer(model.parameters())
learn = Learner(train_dl, valid_dl, model, optimizer = opt)

In [ ]:
learn.fit(10,1e-1)